## Decision Tree from Scratch
### Wyatt Cupp
#### <wyattcupp@gmail.com>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

### Import Data - [Pima Indians Dataset](https://www.kaggle.com/uciml/pima-indians-diabetes-database)

Store data in parallel project directory `decision-tree-classifier/data/`

In [2]:
data = pd.read_csv("./data/diabetes.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
len(data)

768

### Creating Splits - Gini Impurity
For numeric features, we must calculate the various thresholds and corresponding gini scores for each threshold.

See [this link](https://www.analyticsvidhya.com/blog/2021/03/how-to-select-best-split-in-decision-trees-gini-impurity/) to understand Gini Impurity.

In [4]:
from sklearn.model_selection import train_test_split
X = data.drop('Outcome', axis=1, inplace=False)
y = data['Outcome']

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.20, random_state=0)
X_train.shape, y_train.shape

((614, 8), (614,))

### Building the Decision Tree Classifier

In [7]:
class Node:
    '''
    A class representing a Node in a Decision Tree.
    '''
    def __init__(self, gini, num_samples, pred_class):
        self.gini = gini # total gini index score
        self.pred_class = pred_class
        self.num_samples = num_samples
        
        self.feat_index = None
        self.split_threshold=None
        self.left = None
        self.right = None
        
class DecisionTree:
    '''
    A class representing a Decision Tree Classifier, which evaluates split costs
    by using a Gini impurity calculation.
    '''
    def __init__(self, max_depth=None, verbose=False):
        self.max_depth = max_depth
        self.verbose = verbose
        self.total_classes = 0
        self.total_feats = 0
        self.tree = None
        
    def fit(self,X,y): # TODO: detect and convert Pandas DataFrames as input to numpy arrays
        '''
        Fits the input data to the model (grows the tree).
        '''
        self.total_classes = len(set(y))
        self.total_feats = X.shape[1]
        if self.max_depth:
            self.tree = self._grow(X,y,depth=0)
        else:
            self.tree = self._grow(X,y)
         
    def predict(self,X):
        '''
        Predicts all X test samples using the Decision Tree.
        A call to fit() is necessary prior to prediction.
        '''
        preds = []
        
        for row in X:
            curr = self.tree
            while curr.left:
                if row[curr.feat_index] < curr.split_threshold:
                    curr = curr.left
                else: curr = curr.right
            preds.append(curr.pred_class)
        return preds
    
    def info(self): # TODO
        '''
        Print the details of the model.
        '''
        print()
    
    def _grow(self,X,y,depth=None):
        '''
        Recursively grows this DecisionTree and returns the root Node.
        '''
        if self.verbose: 
            if depth is not None:
                print('Current depth: {}'.format(depth))
        node = Node(gini=self._calc_gini(y), num_samples=y.size, pred_class=np.argmax(
            [np.sum(y==c) for c in range(len(set(y)))]))
        
        if depth and depth >= self.max_depth:
            return node
        
        gini, split_idx, split_threshold = self._split(X,y)
        
        if gini < node.gini: # recursively call _grow for left and right child nodes
            node.feat_index = split_idx
            node.split_threshold = split_threshold
            
            left_cond = X[:,split_idx] < split_threshold # left indices, negate for right indices
            node.left = self._grow(X[left_cond], y[left_cond], depth+1 if depth is not None else None)
            node.right = self._grow(X[~left_cond],y[~left_cond], depth+1 if depth is not None else None)
        
        return node
            
    
    def _calc_gini(self,y):
        '''
        Calculates the unweighted gini score for the given data.
        '''
        class_counts = [np.sum(y==c) for c in range(len(set(y)))]
        return 1 - np.sum([(count/len(y))**2 for count in class_counts])
    
    def _split(self,X,y):
        '''
        Calculates the optimal split of the given dataset based on the lowest gini index
        and returns the following:
        
        - gini: Minimum gini score for the given data (out of all the feats)
        - split_idx: The split index (the column index) of the feature selected as a split
        - split_threshold: The numerical threshold value where the split occurs in the data
        '''
        assert(len(X)==len(y)) # ensure data has same length
        if y.size <= 1:
            return 999999, None, None
        
        num_feats = X.shape[1]

        gini, split_idx, split_threshold = 999999, None, None
        for feat in range(num_feats):
            vals, labels = zip(*sorted(zip(X[:,feat], y)))
            vals = np.array(vals)
            labels = np.array(labels)
            
            for index in range(1, len(y)):
                if vals[index-1]==vals[index]: # avoids attempting a split on identical values
                    continue
            
                curr_threshold = (vals[index-1] + vals[index]) / 2 # adjacent mean

                # calculate left and right gini scores
                left = self._calc_gini(labels[:index])
                right = self._calc_gini(labels[index:])

                # calculate total (weighted) gini impurity for left and right gini scores
                curr_gini = ((len(vals[:index]) / y.size)*left) + ((len(vals[index:]) / y.size)*right)
                if curr_gini < gini:
                    gini, split_idx, split_threshold = curr_gini, feat, curr_threshold

        if self.verbose: print("Best Gini: {}\nFeature Index: {}\nThreshold Value: {}\n".format(gini, split_idx, split_threshold))
        return gini, split_idx, split_threshold
        

### Fitting the Data

In [8]:
dt = DecisionTree(max_depth=5, verbose=False)
dt.fit(X_train.to_numpy(), y_train.to_numpy())

### Testing and Metrics

In [9]:
from sklearn.metrics import accuracy_score
y_pred = dt.predict(X=X_test.to_numpy())
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Accuracy Score: 0.7532467532467533


### Compare to Sklearn Decision Tree Classifier

In [10]:
from sklearn.tree import DecisionTreeClassifier

sk_dt = DecisionTreeClassifier(max_depth=5)
sk_dt.fit(X_train.to_numpy(), y_train.to_numpy())
y_pred_sk = sk_dt.predict(X_test.to_numpy())

print("Sklearn Accuracy Score:", accuracy_score(y_test, y_pred_sk))

Sklearn Accuracy Score: 0.7532467532467533
